In [1]:
!pip install openai

     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00

In [1]:
import os
import openai

In [2]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval
import json
import pickle
import time

## PROMPT 0-2

### THIS FIRST BLOCK OF CODE UNTIL TABOO SHOULD BE RUN 3 TIMES TO COLLECT PROMPT_0, PROMPT_1 and PROMPT_2 DATA

In [2]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

# this is the path to the dataset, we dont use taboo words at first
f = open('./taboo_data/robust_data_collection/intent_classification/taboo_1_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [4]:
dct_phrases = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']

In [6]:
default_prompt = 'Rephrase an original question or statement 5 times. Original phrase: "{}"'

In [7]:
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(default_prompt.format(phrase))

In [4]:
openai.api_key = '' # YOUR OPENAI API KEY GOES HERE

In [5]:
def request_response_from_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        #prompt=prompt,
        messages=[
            {"role": "system", "content": "You are a crowdsourcing worker that earns a living through creating paraphrases."},
            {"role": "user", "content": prompt}],
        temperature=1,
        frequency_penalty=0.0,
        presence_penalty=1.5,
        n=13)
    return response

In [6]:
def request_with_checks(prompt):
    success = False
    count = 0
    while not success:
        if count > 0:
            print(f'Retrying with again. Current number of retries: {count}')
        if count >= 5:
            raise Exception('Too many attempts')
        try:
            response = request_response_from_gpt(prompt)
            sucess = True
            break
        except openai.error.RateLimitError as e:
            print(e)
            time.sleep(20)
            count += 1
        except openai.error.APIConnectionError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.APIError or openai.error.JSONDecodeError:
            time.sleep(5)
            count += 1
    return response

In [7]:
dct_responses = {}

In [35]:
# you can adjust the time.sleep duration based on the rate limit of openai
for key in dct_final_prompts:
    print(key)
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(20)

phone
Rephrase an original question or statement 5 times. Original phrase: "what's the phone number of my bank"
Rephrase an original question or statement 5 times. Original phrase: "what number do i call to reach my bank"
Rephrase an original question or statement 5 times. Original phrase: "what's my bank's phone number"
appointment
Rephrase an original question or statement 5 times. Original phrase: "i would like to meet with a bank representative"
Rephrase an original question or statement 5 times. Original phrase: "can i schedule an appointment with someone at the bank?"
Rephrase an original question or statement 5 times. Original phrase: "how do i set up an appointment at the bank?"
hours
Rephrase an original question or statement 5 times. Original phrase: "what time does the bank open?"
Rephrase an original question or statement 5 times. Original phrase: "can you check my bank's hours?"
Rephrase an original question or statement 5 times. Original phrase: "when does the bank close?

In [36]:
with open('responses_0.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)
    
# with open('responses_1.pkl', 'wb') as file:
#     pickle.dump(dct_responses, file)
    
# with open('responses_2.pkl', 'wb') as file:
#     pickle.dump(dct_responses, file)

In [37]:
dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [38]:
chatGpt_0 = pd.DataFrame.from_dict(dct_df)

In [39]:
chatGpt_0.drop_duplicates(subset=['text']).dropna('text').to_csv('chatGPT_0.csv', index=False)
#chatGpt_0.drop_duplicates(subset=['text']).dropna('text').to_csv('chatGPT_1.csv', index=False)
#chatGpt_0.drop_duplicates(subset=['text']).dropna('text').to_csv('chatGPT_2.csv', index=False)

## TABOO 1

In [70]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('./taboo_data/robust_data_collection/intent_classification/taboo_1_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [71]:
dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

In [73]:
defaul_taboo_prompt = 'Rephrase an original question or statement 5 times. Original phrase: "{}". Don’t use the words “{}”, "{}" or “{}” in your responses.'

In [74]:
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2]))

In [76]:
dct_responses = {}

In [77]:
for key in dct_final_prompts:
    print(key)
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(20)

phone
Rephrase an original question or statement 5 times. Original phrase: "what's the phone number of my bank". Don’t use the words “number”, "call" or “contact” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "what number do i call to reach my bank". Don’t use the words “number”, "call" or “contact” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "what's my bank's phone number". Don’t use the words “number”, "call" or “contact” in your responses.
appointment
Rephrase an original question or statement 5 times. Original phrase: "i would like to meet with a bank representative". Don’t use the words “appointment”, "representative" or “meeting” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "can i schedule an appointment with someone at the bank?". Don’t use the words “appointment”, "representative" or “meeting” in your responses.
Rephrase an original question or statem

In [78]:
with open('responses_taboo_1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [79]:
dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [80]:
chatGpt_1 = pd.DataFrame.from_dict(dct_df)

In [81]:
chatGpt_1.drop_duplicates(subset=['text']).dropna('text').to_csv('chatGPT_taboo_1.csv', index=False)

## TABOO 2

In [8]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('./taboo_data/robust_data_collection/intent_classification/taboo_2_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [9]:
dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

In [10]:
defaul_taboo_prompt = 'Rephrase an original question or statement 5 times. Original phrase: "{}". Don’t use the words “{}”, "{}", "{}", "{}", "{}" or “{}” in your responses.'

In [11]:
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3], dct_taboo[key][4], dct_taboo[key][5]))

In [17]:
%%time

for key in dct_final_prompts:
    print(key)
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(20)

phone
Rephrase an original question or statement 5 times. Original phrase: "what's the phone number of my bank". Don’t use the words “number”, "call", "contact", "phone", "reach" or “connect” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "what number do i call to reach my bank". Don’t use the words “number”, "call", "contact", "phone", "reach" or “connect” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "what's my bank's phone number". Don’t use the words “number”, "call", "contact", "phone", "reach" or “connect” in your responses.
appointment
Rephrase an original question or statement 5 times. Original phrase: "i would like to meet with a bank representative". Don’t use the words “appointment”, "representative", "meeting", "meet", "consultation" or “discuss” in your responses.
Rephrase an original question or statement 5 times. Original phrase: "can i schedule an appointment with someone at the bank?". D

In [18]:
with open('responses_taboo_2.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [2]:
dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [4]:
import pandas as pd
chatGpt_2 = pd.DataFrame.from_dict(dct_df)

In [5]:
chatGpt_2.to_csv('chatGPT_taboo_2.csv', index=False)

## Prepare the datasets - combine the collected rounds into final prompt and taboo datasets

In [7]:
dct = {'phone': 0,
 'appointment': 1,
 'hours': 2,
 'safe': 3,
 'checks': 4,
 'location': 5,
 'close': 6,
 'withdrawal': 7,
 'routing': 8,
 'balance': 9}

def function(x):
    return dct[x]

In [6]:
same_0_gpt = pd.read_csv('chatGPT_0.csv').dropna(subset=['text'])
same_1_gpt = pd.read_csv('chatGPT_1.csv').dropna(subset=['text'])
same_2_gpt = pd.read_csv('chatGPT_2.csv').dropna(subset=['text'])

In [8]:
same_0_gpt['label'] = same_0_gpt['intent'].apply(function)
same_0_gpt = same_0_gpt.sample(frac=1).reset_index(drop=True)

same_1_gpt['label'] = same_1_gpt['intent'].apply(function)
same_1_gpt = same_1_gpt.sample(frac=1).reset_index(drop=True)

same_2_gpt['label'] = same_2_gpt['intent'].apply(function)
same_2_gpt = same_2_gpt.sample(frac=1).reset_index(drop=True)

In [16]:
same_gpt_12 = same_1_gpt.append(same_2_gpt).drop_duplicates(subset=['text']).dropna(subset=['text'])

In [17]:
same_gpt_12[['text', 'label']].to_csv('gpt_datasets/prompt_gpt_12.csv', index=False)

In [18]:
same_0_gpt[['text', 'label']].drop_duplicates(subset=['text']).dropna(subset=['text']).to_csv('gpt_datasets/prompt_gpt_0.csv', index=False)

In [25]:
taboo_1_gpt = pd.read_csv('chatGPT_taboo_1.csv').dropna(subset=['text'])
taboo_2_gpt = pd.read_csv('chatGPT_taboo_2.csv').dropna(subset=['text'])

In [35]:
taboo_1_gpt['label'] = taboo_1_gpt['intent'].apply(function)
taboo_1_gpt = taboo_1_gpt.sample(frac=1).reset_index(drop=True)

taboo_2_gpt['label'] = taboo_2_gpt['intent'].apply(function)
taboo_2_gpt = taboo_2_gpt.sample(frac=1).reset_index(drop=True)

In [21]:
taboo_gpt_12 = taboo_1_gpt.append(taboo_2_gpt).drop_duplicates(subset=['text']).dropna(subset=['text'])

In [22]:
taboo_gpt_12[['text', 'label']].to_csv('gpt_datasets/taboo_gpt_12.csv', index=False)

In [36]:
# Now the filtering for taboo words

f = open('./taboo_data/robust_data_collection/intent_classification/taboo_1_config.json')
data = json.load(f)
f.close()

dct_taboo = {}
for key in data:
    dct_taboo[key] = data[key]['avoid_words']

def are_taboo_words_in(x, y, dct_taboo_w):
    words = x.replace('"', '').split()
    forb_words = dct_taboo_w[y]
    for word in forb_words:
        if word in words:
            return False
    return True

taboo_1_gpt['no_taboo_words'] = taboo_1_gpt.apply(lambda x: are_taboo_words_in(x['text'], x['intent'], dct_taboo), axis=1)
taboo_1_gpt = taboo_1_gpt[taboo_1_gpt['no_taboo_words'] == True]

f = open('./taboo_data/robust_data_collection/intent_classification/taboo_2_config.json')
data = json.load(f)
f.close()

dct_taboo = {}
for key in data:
    dct_taboo[key] = data[key]['avoid_words']

taboo_2_gpt['no_taboo_words'] = taboo_2_gpt.apply(lambda x: are_taboo_words_in(x['text'], x['intent'], dct_taboo), axis=1)
taboo_2_gpt = taboo_2_gpt[taboo_2_gpt['no_taboo_words'] == True]

In [37]:
taboo_gpt_12_filt = taboo_1_gpt.append(taboo_2_gpt).drop_duplicates(subset=['text']).dropna(subset=['text'])

In [38]:
taboo_gpt_12_filt[['text', 'label']].to_csv('gpt_datasets/taboo_gpt_12_filt.csv', index=False) # contains taboo words

## NOW THE DATA PREP FROM THE LARSON ET AL. 2020 STUDY

### These csv files were created by changing the existing jsons from the study into csv files, I recommend skipping this and doing it on your own

Keep in mind, that the prompt 0 split is contained in all the datasets, so the 1 and 2 splits for both taboo and prompt methods should be kept separately

In [48]:
same_0 = pd.read_csv('larson_same_0.csv')
same_0['label'] = same_0['intent'].apply(function)
same_0 = same_0.sample(frac=1).reset_index(drop=True)

taboo_1 = pd.read_csv('larson_taboo_1.csv')
taboo_2 = pd.read_csv('larson_taboo_2.csv')

taboo_1['label'] = taboo_1['intent'].apply(function)
taboo_2['label'] = taboo_2['intent'].apply(function)

In [49]:
taboo_h = taboo_1.append(taboo_2).drop_duplicates(subset=['text']).dropna(subset=['text'])

In [50]:
taboo_h[['text', 'label']].to_csv('gpt_datasets/taboo_h_12.csv', index=False) # contains taboo words

In [51]:
same_0[['text', 'label']].to_csv('gpt_datasets/prompt_0_h.csv', index=False)

In [52]:
same_1 = pd.read_csv('larson_same_1.csv')
same_1['label'] = same_1['intent'].apply(function)
same_1 = same_1.sample(frac=1).reset_index(drop=True)

same_2 = pd.read_csv('larson_same_2.csv')
same_2['label'] = same_2['intent'].apply(function)
same_2 = same_2.sample(frac=1).reset_index(drop=True)

In [53]:
same_h = same_1.append(same_2).drop_duplicates(subset=['text']).dropna(subset=['text'])

In [54]:
same_h[['text', 'label']].to_csv('gpt_datasets/same_h_12.csv', index=False) # contains taboo words